In [7]:
#Importing packages 
import fastf1 as ff1
import numpy as np
import pandas as pd

#Enabling cache
ff1.Cache.enable_cache('/Users/username/opt/anaconda3/lib/python3.9/site-packages/fastf1/cache/')

#Adjusting display parameters for dataframe outputs
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)

In [16]:
#Querying data from FastF1 API (for Qualifying round) and loading API data for qualifying results into pandas dataframe
quali = ff1.get_session(2022, 1, 'Q')
quali.load()
dfQ = pd.DataFrame(quali.results)

#Creating new dataframe, excluding unnecessary columns
for index, row in dfQ.iterrows():
    quali_list = {}
    quali_list['FullName'] = dfQ['FullName']
    quali_list['Abbreviation'] = dfQ['Abbreviation']
    quali_list['TeamName'] = dfQ['TeamName']
    quali_list['Position'] = dfQ['Position']
    quali_list['Q1'] = dfQ['Q1']
    quali_list['Q2'] = dfQ['Q2']
    quali_list['Q3'] = dfQ['Q3']
    quali_list['Status'] = dfQ['Status'] 
df_quali = pd.DataFrame(quali_list)
print(df_quali)

core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '1', '55', '11', '44', '77', '20', '14', '63', '10', '31', '47', '4', '23', '24', '22', '27', '3', '18', '6']


            FullName Abbreviation         TeamName  Position                     Q1                     Q2  \
16   Charles Leclerc          LEC          Ferrari       1.0 0 days 00:01:31.471000 0 days 00:01:30.932000   
1     Max Verstappen          VER  Red Bull Racing       2.0 0 days 00:01:31.785000 0 days 00:01:30.757000   
55      Carlos Sainz          SAI          Ferrari       3.0 0 days 00:01:31.567000 0 days 00:01:30.787000   
11      Sergio Perez          PER  Red Bull Racing       4.0 0 days 00:01:32.311000 0 days 00:01:31.008000   
44    Lewis Hamilton          HAM         Mercedes       5.0 0 days 00:01:32.285000 0 days 00:01:31.048000   
77   Valtteri Bottas          BOT       Alfa Romeo       6.0 0 days 00:01:31.919000 0 days 00:01:31.717000   
20   Kevin Magnussen          MAG     Haas F1 Team       7.0 0 days 00:01:31.955000 0 days 00:01:31.461000   
14   Fernando Alonso          ALO           Alpine       8.0 0 days 00:01:32.346000 0 days 00:01:31.621000   
63    Geor

In [17]:
#Creating calculation for 'Did Not Qualify' score
df_quali['s_DNQ'] = np.where(pd.isnull(df_quali['Q1']) == True, -5, 0)

#Creating calculations for 'Q1', 'Q2', and 'Q3' scores
df_quali['s_Q1'] = np.where(pd.notnull(df_quali['Q1']) == True, 1, 0)
df_quali['s_Q2'] = np.where(pd.notnull(df_quali['Q2']) == True, 2, 0)
df_quali['s_Q3'] = np.where(pd.notnull(df_quali['Q3']) == True, 3, 0)

#Creating calculation for 'Disqualified' score (Qualifying round)
df_quali['s_DQ'] = np.where(df_quali['Status'] == 'Disqualified', -10, 0)

#Creating new column for 'Teammate'
df_quali['Teammate'] = np.where(df_quali['Abbreviation'] == 'TSU', 'GAS',
                        np.where(df_quali['Abbreviation'] == 'SAI', 'LEC',
                        np.where(df_quali['Abbreviation'] == 'LEC', 'SAI',
                        np.where(df_quali['Abbreviation'] == 'RIC', 'NOR',
                        np.where(df_quali['Abbreviation'] == 'OCO', 'ALO',
                        np.where(df_quali['Abbreviation'] == 'ALO', 'OCO',
                        np.where(df_quali['Abbreviation'] == 'RUS', 'HAM',
                        np.where(df_quali['Abbreviation'] == 'ZHO', 'BOT',
                        np.where(df_quali['Abbreviation'] == 'MAG', 'MSC',
                        np.where(df_quali['Abbreviation'] == 'STR', 'VET',
                        np.where(df_quali['Abbreviation'] == 'NOR', 'RIC',
                        np.where(df_quali['Abbreviation'] == 'HAM', 'RUS',
                        np.where(df_quali['Abbreviation'] == 'VER', 'PER',
                        np.where(df_quali['Abbreviation'] == 'MSC', 'MAG',
                        np.where(df_quali['Abbreviation'] == 'LAT', 'ALB',
                        np.where(df_quali['Abbreviation'] == 'GAS', 'TSU',
                        np.where(df_quali['Abbreviation'] == 'VET', 'STR',
                        np.where(df_quali['Abbreviation'] == 'PER', 'VER',
                        np.where(df_quali['Abbreviation'] == 'BOT', 'ZHO',
                        np.where(df_quali['Abbreviation'] == 'ALB', 'LAT', ''))))))))))))))))))))

#Creating new dataframe for 'Teammate Qualifying Position'
df_teammateQ = pd.DataFrame(df_quali, columns = ['Abbreviation', 'Position'])
df_teammateQ.columns = ['Teammate', 'Position']

#Merging df_quali and df_teammateQ dataframes using 'Teammate' column as merge key 
df_quali.merge(df_teammateQ, left_on = 'Teammate', right_on = 'Teammate') 
df_quali2 = pd.merge(df_quali, df_teammateQ, on = 'Teammate', how = 'left')

#Creating calculation for 'Qualified Ahead of Teammate' score
df_quali2['s_Teammate'] = np.where(df_quali2['Position_x'] < df_quali2['Position_y'], 2, 0)

#Creating calculation for 'Qualifying Bonus' score
df_quali2['s_QualiBonus'] = np.where(df_quali2['Position_x'] == 1, 5,
                            np.where(df_quali2['Position_x'] == 2, 4,
                            np.where(df_quali2['Position_x'] == 3, 3,
                            np.where(df_quali2['Position_x'] == 4, 2,
                            np.where(df_quali2['Position_x'] == 5, 1,
                            np.where(df_quali2['Position_x'] > 5, 0, 0))))))

#Creating final dataframe for Qualifying round scoring
for index, row in df_quali2.iterrows():
    quali_results = {}
    quali_results['FullName'] = df_quali2['FullName']
    quali_results['Driver'] = df_quali2['Abbreviation']
    quali_results['Team'] = df_quali2['TeamName']
    quali_results['Position'] = df_quali2['Position_x']
    quali_results['Q1'] = df_quali2['Q1']
    quali_results['Q2'] = df_quali2['Q2']
    quali_results['Q3'] = df_quali2['Q3']
    quali_results['Status'] = df_quali2['Status']
    quali_results['TeammatePosition'] = df_quali2['Position_y']
    quali_results['s_DNQ'] = df_quali2['s_DNQ']
    quali_results['s_Q1'] = df_quali2['s_Q1']
    quali_results['s_Q2'] = df_quali2['s_Q2']
    quali_results['s_Q3'] = df_quali2['s_Q3']
    quali_results['s_Teammate'] = df_quali2['s_Teammate']
    quali_results['s_DQ'] = df_quali2['s_DQ']
    quali_results['s_QualiBonus'] = df_quali2['s_QualiBonus'] 
df_quali_results = pd.DataFrame(quali_results)

#Creating new column that sums each driver's total score for the Qualifying round, excluding their 'Qualifying Bonus'
df_quali_results['s_QualiTotal'] = df_quali_results['s_DNQ'] + df_quali_results['s_Q1'] + df_quali_results['s_Q2'] + df_quali_results['s_Q3'] + df_quali_results['s_DQ'] +  df_quali_results['s_Teammate']

#Exporting final dataframe for Qualifying round to csv
df_quali_results.to_csv('Qualifying-1-results.csv')
print(df_quali_results)

            FullName Driver             Team  Position                     Q1                     Q2  \
0    Charles Leclerc    LEC          Ferrari       1.0 0 days 00:01:31.471000 0 days 00:01:30.932000   
1     Max Verstappen    VER  Red Bull Racing       2.0 0 days 00:01:31.785000 0 days 00:01:30.757000   
2       Carlos Sainz    SAI          Ferrari       3.0 0 days 00:01:31.567000 0 days 00:01:30.787000   
3       Sergio Perez    PER  Red Bull Racing       4.0 0 days 00:01:32.311000 0 days 00:01:31.008000   
4     Lewis Hamilton    HAM         Mercedes       5.0 0 days 00:01:32.285000 0 days 00:01:31.048000   
5    Valtteri Bottas    BOT       Alfa Romeo       6.0 0 days 00:01:31.919000 0 days 00:01:31.717000   
6    Kevin Magnussen    MAG     Haas F1 Team       7.0 0 days 00:01:31.955000 0 days 00:01:31.461000   
7    Fernando Alonso    ALO           Alpine       8.0 0 days 00:01:32.346000 0 days 00:01:31.621000   
8     George Russell    RUS         Mercedes       9.0 0 days 00

In [18]:
#Querying data from FastF1 API (for Race round) and loading API data for race results into pandas dataframe
race = ff1.get_session(2022, 1, 'R')
race.load()
dfR = pd.DataFrame(race.results)

#Loading lap data for each driver into dataframe
dfL = pd.DataFrame(race.laps)
Laps = dfL['Driver'].value_counts()
dfLaps = pd.DataFrame(Laps)
dfLaps['Abbreviation'] = dfLaps.index

#Creating new dataframe, excluding unnecessary columns
for index, row in dfR.iterrows():
    race_list = {}
    race_list['Abbreviation'] = dfR['Abbreviation']
    race_list['Name'] = dfR['FullName']
    race_list['TeamName'] = dfR['TeamName']
    race_list['Grid'] = dfR['GridPosition']
    race_list['Position'] = dfR['Position']
    race_list['Status'] = dfR['Status']
df_race = pd.DataFrame(race_list)

#Merging df_race and dfLaps dataframes
df_race_lookup = pd.merge(df_race, dfLaps)

#Creating new column to identify driver with the 'Fastest Lap' of the race
fastestlap = race.laps.pick_fastest(only_by_time=False)

df_race_lookup.loc[df_race_lookup['Abbreviation'] == fastestlap['Driver'], 'FastestLapRank'] = 1 
df_race_lookup.loc[df_race_lookup['Abbreviation'] != fastestlap['Driver'], 'FastestLapRank'] = 'N/A'

#Creating new dataframe, including count of completed laps by each driver
for index, row in df_race_lookup.iterrows():
    race_table = {}
    race_table['Abbreviation'] = df_race_lookup['Abbreviation']
    race_table['Name'] = df_race_lookup['Name']
    race_table['TeamName'] = df_race_lookup['TeamName']
    race_table['Grid'] = df_race_lookup['Grid']
    race_table['Position'] = df_race_lookup['Position']
    race_table['Status'] = df_race_lookup['Status']
    race_table['FastestLap'] = df_race_lookup['FastestLapRank']
    race_table['Laps'] = df_race_lookup['Driver']
df_race_final = pd.DataFrame(race_table)

core           INFO 	Loading data for Bahrain Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '55', '44', '63', '20', '77', '31', '22', '14', '24', '47', '18', '23', '3', '4', '6', '27', '11', '1', '10']


In [19]:
#Creating calculation for number of positions gained/lost during Race round 

df_race_final['Gained_Lost'] = df_race_final['Grid'] - df_race_final['Position']

#Creating calculation for 'Positions Gained' score
df_race_final['Gained_Lost_x2'] = df_race_final['Gained_Lost'] * 2

df_race_final['s_Gained'] = np.where(df_race_final['Gained_Lost_x2'] > 10, 10, 
                            np.where(df_race_final['Gained_Lost_x2'] < 0, 0, 
                            df_race_final['Gained_Lost_x2']))

#Creating calculation for 'Positions Lost' score
df_race_final['s_Lost'] = np.where(df_race_final['Gained_Lost_x2'] < -10, -10, 
                            np.where(df_race_final['Gained_Lost_x2'] > 0, 0, 
                            df_race_final['Gained_Lost_x2']))

#Creating calculation calculation for 'Finished Race' score
df_race_final['s_Finished'] = np.where(df_race_final['Status'] == 'Finished', 1, 0)

#Creating calculation for 'Not Classified' score
df_race_final['s_NotClassified'] = np.where(df_race_final['Laps'] <= (df_race_final['Laps'].max() * .9), -10, 0) 

#Creating calculation for 'Fastest Lap' score
df_race_final['s_FastestLap'] = np.where(df_race_final['FastestLap'] == 1, 5, 0)

#Creating calculation for 'Disqualified' score (Race round)
df_race_final['s_DQ'] = np.where(df_race_final['Status'] == 'Disqualified', -20, 0)

#Creating new column for 'Teammate'
df_race_final['Teammate'] = np.where(df_race_final['Abbreviation'] == 'ALB', 'LAT',
                            np.where(df_race_final['Abbreviation'] == 'SAI', 'LEC',
                            np.where(df_race_final['Abbreviation'] == 'LEC', 'SAI',
                            np.where(df_race_final['Abbreviation'] == 'RIC', 'NOR',
                            np.where(df_race_final['Abbreviation'] == 'OCO', 'ALO',
                            np.where(df_race_final['Abbreviation'] == 'ALO', 'OCO',
                            np.where(df_race_final['Abbreviation'] == 'RUS', 'HAM',
                            np.where(df_race_final['Abbreviation'] == 'ZHO', 'BOT',
                            np.where(df_race_final['Abbreviation'] == 'MAG', 'MSC',
                            np.where(df_race_final['Abbreviation'] == 'STR', 'VET',
                            np.where(df_race_final['Abbreviation'] == 'NOR', 'RIC',
                            np.where(df_race_final['Abbreviation'] == 'HAM', 'RUS',
                            np.where(df_race_final['Abbreviation'] == 'VER', 'PER',
                            np.where(df_race_final['Abbreviation'] == 'MSC', 'MAG',
                            np.where(df_race_final['Abbreviation'] == 'LAT', 'ALB',
                            np.where(df_race_final['Abbreviation'] == 'GAS', 'TSU',
                            np.where(df_race_final['Abbreviation'] == 'VET', 'STR',
                            np.where(df_race_final['Abbreviation'] == 'PER', 'VER',
                            np.where(df_race_final['Abbreviation'] == 'BOT', 'ZHO',
                            np.where(df_race_final['Abbreviation'] == 'TSU', 'GAS', ''))))))))))))))))))))

#Creating new dataframe for 'Teammate Race Position'
df_teammate = pd.DataFrame(df_race_final, columns = ['Abbreviation', 'Position'])
df_teammate.columns = ['Teammate', 'Position']

#Merging df_race_final and df_teammate dataframes using 'Teammate' column as merge key 
df_race_final.merge(df_teammate, left_on = 'Teammate', right_on = 'Teammate') 
df_race_final2 = pd.merge(df_race_final, df_teammate, on = 'Teammate', how = 'inner')

#Creating calculation for 'Finished Race Ahead of Teammate' score
df_race_final2['s_Teammate'] = np.where(df_race_final2['Position_x'] < df_race_final2['Position_y'], 3, 0)

#Creating calculation for 'Race Bonus' score
df_race_final2['s_RaceBonus'] = np.where(df_race_final2['Position_x'] == 1, 25, 
                                np.where(df_race_final2['Position_x'] == 2, 22, 
                                np.where(df_race_final2['Position_x'] == 3, 20, 
                                np.where(df_race_final2['Position_x'] == 4, 18, 
                                np.where(df_race_final2['Position_x'] == 5, 16, 
                                np.where(df_race_final2['Position_x'] == 6, 15, 
                                np.where(df_race_final2['Position_x'] == 7, 14, 
                                np.where(df_race_final2['Position_x'] == 8, 13, 
                                np.where(df_race_final2['Position_x'] == 9, 12, 
                                np.where(df_race_final2['Position_x'] == 10, 11, 
                                np.where(df_race_final2['Position_x'] == 11, 10, 
                                np.where(df_race_final2['Position_x'] == 12, 9, 
                                np.where(df_race_final2['Position_x'] == 13, 8, 
                                np.where(df_race_final2['Position_x'] == 14, 7, 
                                np.where(df_race_final2['Position_x'] == 15, 6, 
                                np.where(df_race_final2['Position_x'] == 16, 5, 
                                np.where(df_race_final2['Position_x'] == 17, 4, 
                                np.where(df_race_final2['Position_x'] == 18, 3, 
                                np.where(df_race_final2['Position_x'] == 19, 2, 
                                np.where(df_race_final2['Position_x'] == 20, 1, 0))))))))))))))))))))

#Creating final dataframe for Race round scoring
for index, row in df_race_final2.iterrows():
    race_results = {}
    race_results['Name'] = df_race_final2['Name']
    race_results['Driver'] = df_race_final2['Abbreviation']
    race_results['Team'] = df_race_final2['TeamName']
    race_results['Grid'] = df_race_final2['Grid']
    race_results['Position'] = df_race_final2['Position_x']
    race_results['Status'] = df_race_final2['Status']
    race_results['FastestLap'] = df_race_final2['FastestLap']
    race_results['Laps'] = df_race_final2['Laps']
    race_results['TeammatePosition'] = df_race_final2['Position_y']
    race_results['s_Finished'] = df_race_final2['s_Finished']
    race_results['s_FastestLap'] = df_race_final2['s_FastestLap']
    race_results['s_Teammate'] = df_race_final2['s_Teammate']
    race_results['s_Gained'] = df_race_final2['s_Gained']
    race_results['s_Lost'] = df_race_final2['s_Lost']
    race_results['s_NotClassified'] = df_race_final2['s_NotClassified']
    race_results['s_DQ'] = df_race_final2['s_DQ']
    race_results['s_RaceBonus'] = df_race_final2['s_RaceBonus']
df_race_results = pd.DataFrame(race_results)

#Creating new column that sums each driver's total score for the Race round, excluding their 'Race Bonus'
df_race_results['s_RaceTotal'] = df_race_results['s_Gained'] + df_race_results['s_Lost'] + df_race_results['s_Finished'] + df_race_results['s_NotClassified'] + df_race_results['s_FastestLap'] + df_race_results['s_DQ'] + df_race_results['s_Teammate'] 

#Exporting final dataframe for Race round to csv
df_race_results.to_csv('Race-1-results.csv')    
print(df_race_results)

                Name Driver             Team  Grid  Position         Status FastestLap  Laps  TeammatePosition  \
0    Charles Leclerc    LEC          Ferrari   1.0       1.0       Finished        1.0    57               2.0   
1       Carlos Sainz    SAI          Ferrari   3.0       2.0       Finished        N/A    57               1.0   
2     Lewis Hamilton    HAM         Mercedes   5.0       3.0       Finished        N/A    57               4.0   
3     George Russell    RUS         Mercedes   9.0       4.0       Finished        N/A    57               3.0   
4    Kevin Magnussen    MAG     Haas F1 Team   7.0       5.0       Finished        N/A    57              11.0   
5    Valtteri Bottas    BOT       Alfa Romeo   6.0       6.0       Finished        N/A    57              10.0   
6       Esteban Ocon    OCO           Alpine  11.0       7.0       Finished        N/A    57               9.0   
7       Yuki Tsunoda    TSU       AlphaTauri  16.0       8.0       Finished        N/A  

In [20]:
#Creating initial dataframe for Constructor scoring
teams = {'Team' : ['Alfa Romeo', 'AlphaTauri', 'Alpine', 'Aston Martin', 'Ferrari', 'Haas F1 Team', 'McLaren', 'Mercedes', 'Red Bull Racing', 'Williams']}
df_constructors = pd.DataFrame(teams)

#Creating calculation for each constructor's 'Qualifying Total' score
s_Quali = df_quali_results.groupby('Team')['s_QualiTotal'].sum() - 2
df_s_Quali = pd.DataFrame(s_Quali)
df_constructors1 = pd.merge(df_constructors, df_s_Quali, on = 'Team', how = 'inner')

#Creating calculation for each constructor's 'Qualifying Bonus' score
s_QualiBonus = df_quali_results.groupby('Team')['s_QualiBonus'].sum()
df_s_QualiBonus = pd.DataFrame(s_QualiBonus)
df_constructors2 = pd.merge(df_constructors1, df_s_QualiBonus, on = 'Team', how = 'inner')

#Creating calculation for each constructor's 'Race Total' score
s_Race = df_race_results.groupby('Team')['s_RaceTotal'].sum() - 3
df_s_Race = pd.DataFrame(s_Race)
df_constructors3 = pd.merge(df_constructors2, df_s_Race, on = 'Team', how = 'inner')

#Creating calculation for each constructor's 'Race Bonus' score
s_RaceBonus = df_race_results.groupby('Team')['s_RaceBonus'].sum()
df_s_RaceBonus = pd.DataFrame(s_RaceBonus)
df_constructors4 = pd.merge(df_constructors3, df_s_RaceBonus, on = 'Team', how = 'inner')

#Exporting final dataframe for Constructor scoring to csv
df_constructors4.to_csv('Constructor-1-results.csv')    
print(df_constructors4)

              Team  s_QualiTotal  s_QualiBonus  s_RaceTotal  s_RaceBonus
0       Alfa Romeo             9             0         12.0           26
1       AlphaTauri             7             0         -9.0           14
2           Alpine             9             0          8.0           26
3          Ferrari            12             8          9.0           47
4     Haas F1 Team             9             0          8.0           26
5          McLaren             4             0          6.0           13
6         Mercedes            12             1         16.0           38
7  Red Bull Racing            12             6        -20.0            5
8         Williams             4             0         12.0           13
